<a href="https://colab.research.google.com/github/BishalMondal/Crop-Disease-Detection/blob/main/ResNext_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from google.colab import drive
from tensorflow.keras.applications.resnet50 import preprocess_input
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import interp
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score, roc_curve, plot_roc_curve
from sklearn.metrics import confusion_matrix, classification_report, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D, concatenate, BatchNormalization, Activation, add
from keras.layers import Conv2D, MaxPooling2D, Reshape, Flatten, Dense
from keras.models import Model, model_from_json
from tensorflow.keras.optimizers import Adam
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.callbacks import EarlyStopping, ModelCheckpoint
sns.set_theme(style="whitegrid")

In [ ]:
%run /content/gdrive/MyDrive/CNN_Models/Crop_ResNeXt_1.ipynb

In [ ]:
%run /content/gdrive/MyDrive/data_loader/Crop_Data_Loader.ipynb

In [ ]:
image_resize = 224
batch_size_training = 64
batch_size_validation = 64
train_generator = loadTrainData(image_resize, batch_size_training)
validation_generator = loadValidationData(image_resize, batch_size_validation)

Found 30389 images belonging to 61 classes.
Found 4534 images belonging to 61 classes.


##ResNext Model Load

In [ ]:
# Configurations
length = 224
width = 224
model_name = 'ResNeXt'  
model_width = 16  # Width of the Initial Layer, 
num_channel = 3  # Number of Channels in the Model
problem_type = 'Classification' # Classification or Regression
output_nums = 61  # Number of Class for Classification Problems, 
# Build, Compile and Print Summary
Model = ResNeXt(length, width, num_channel, model_width, cardinality=8, problem_type=problem_type, output_nums=output_nums, pooling='avg', dropout_rate=False).ResNeXt152()
Model.compile(loss='categorical_crossentropy', optimizer='adam', metrics= ['mean_squared_error', 'accuracy'])
Model.summary()

Streaming output truncated to the last 5000 lines.
 conv2d_296 (Conv2D)            (None, 14, 14, 64)   576         ['lambda_129[0][0]']             
                                                                                                  
 conv2d_298 (Conv2D)            (None, 14, 14, 64)   576         ['lambda_130[0][0]']             
                                                                                                  
 conv2d_300 (Conv2D)            (None, 14, 14, 64)   576         ['lambda_131[0][0]']             
                                                                                                  
 conv2d_302 (Conv2D)            (None, 14, 14, 64)   576         ['lambda_132[0][0]']             
                                                                                                  
 conv2d_304 (Conv2D)            (None, 14, 14, 64)   576         ['lambda_133[0][0]']             
                                                          

## Features Extraction

In [ ]:
import os, shutil
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 2048))  
    # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count, 61))
    # Preprocess data
    generator = datagen.flow_from_directory(directory,
                                            target_size=(img_width,img_height),
                                            batch_size = batch_size,
                                            class_mode='categorical')
    
    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels
    
train_features, train_labels = extract_features(train_dir, 30389)  
validation_features, validation_labels = extract_features(valid_dir, 4534)

In [ ]:
train_labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
nsamples1, nx1, ny1 ,nz1= train_features.shape
d2_train_dataset = train_features.reshape((nsamples1,nx1*ny1*nz1))

In [ ]:
nsamples, nx, ny ,nz= validation_features.shape
d2_test_dataset = validation_features.reshape((nsamples,nx*ny*nz))

### Implementing a ResNext

In [ ]:
#Model training
history = Model.fit(train_generator,
                    steps_per_epoch=batch_size_training/4, 
                    epochs=300, 
                    batch_size=batch_size_training, 
                    verbose=1, 
                    validation_data=validation_generator,
                    validation_steps=batch_size_validation/4, 
                    shuffle=True )
                    callbacks=callbacks)

In [ ]:
y_pred=model.predict(d2_test_dataset)
y_pred

array([0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
       1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1.,
       0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
       0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
       1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0.

In [ ]:
print('Model accuracy score with default hyperparameters: {0:0.4f}' . format(accuracy_score(validation_labels, y_pred)))

Model accuracy score with default hyperparameters: 0.9378


In [ ]:
accuracy_score(y_pred,validation_labels)
print(classification_report(y_pred,validation_labels))

              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94       232
         1.0       0.94      0.94      0.94       234

    accuracy                           0.94       466
   macro avg       0.94      0.94      0.94       466
weighted avg       0.94      0.94      0.94       466



In [ ]:
confusion_matrix(y_pred,validation_labels)

array([[218,  14],
       [ 15, 219]])